In [ ]:
!pip install langchain langchain-community langchain langchain-openai langchain-anthropic chromadb gitpython

# [4주차] 심화과제: 코드 리뷰 LLM 서비스
---

# [MY CODE] 1. 환경 변수 로드

In [437]:
from dotenv import load_dotenv
import os

load_dotenv()

class Config:
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    ANTHROPIC_API_KEY = os.environ['ANTHROPIC_API_KEY']
    PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
    CHROMA_DB_PATH = os.environ['CHROMA_DB_PATH']

---
# [MY CODE] 2. LLM(OpenAI, Anthropic)

In [438]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=Config.OPENAI_API_KEY, max_tokens=7000)
anthropic_llm = ChatAnthropic(model_name="claude-3-5-sonnet-20241022", api_key=Config.ANTHROPIC_API_KEY, max_tokens=7000)

---
# [MY CODE] 3. Embedding(OpenAI)

In [439]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=Config.OPENAI_API_KEY)

---
# [MY CODE] 4. VectorStore(ChromaDB)

In [440]:
from langchain.vectorstores import Chroma

vector_store = Chroma(persist_directory=Config.CHROMA_DB_PATH, embedding_function=embeddings)

---
# [MY CODE] 5. Github repository -> Store

In [441]:
from git import Repo
import hashlib

# GitHub 레포지토리를 로컬로 클론
def clone_github_repo(repo_url, local_path):
    if os.path.exists(local_path):
        print("로컬에 이미 레포지토리가 존재합니다. 변경 사항을 pull합니다.")
        repo = Repo(local_path)
        repo.remotes.origin.pull()
    else:
        print(f"Cloning {repo_url} into {local_path}...")
        Repo.clone_from(repo_url, local_path)
    print("레포지토리 클론 또는 업데이트 완료!")

# 레포지토리 내 Python 파일 경로를 찾기
def find_python_files(repo_path):
    python_files = []
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files

# 파일의 해시값(SHA256)을 계산
def calculate_file_hash(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

# 파일이 업데이트되었는지 확인
def is_file_updated(file_path, vector_store):
    file_hash = calculate_file_hash(file_path)
    # Chroma에서 해당 파일의 메타데이터 검색
    results = vector_store.similarity_search(file_path, k=1, filter={"file_path": file_path})
    if results:
        # 저장된 해시값과 비교
        return results[0].metadata.get("file_hash") != file_hash
    return True

# Python 파일을 읽어 벡터화하고 Chroma DB에 저장
def store_python_files_in_chroma(file_paths, vector_store):
    for file_path in file_paths:
        try:
            if not is_file_updated(file_path, vector_store):
                print(f"파일이 변경되지 않았습니다. 건너뜁니다: {file_path}")
                continue

            with open(file_path, "r", encoding="utf-8") as file:
                code_content = file.read()

            # 파일 메타데이터 생성
            metadata = {
                "file_path": file_path,
                "file_hash": calculate_file_hash(file_path)
            }

            # 벡터 스토어에 추가
            vector_store.add_texts([code_content], metadatas=[metadata])
            print(f"{file_path} 저장 완료!")
        except Exception as e:
            print(f"파일 처리 중 오류 발생: {file_path}, 오류: {e}")
    vector_store.persist()
    print("Chroma DB에 모든 파일 저장 완료!")

# Chroma DB에 저장된 모든 코드 파일의 메타데이터와 내용을 출력합니다.
def list_all_files_with_content(vector_store):
    try:
        results = vector_store.get(include=["metadatas", "documents"])

        print("저장된 코드 파일과 내용:")
        for i, (metadata, document) in enumerate(zip(results["metadatas"], results["documents"]), 1):
            file_path = metadata.get("file_path", "Unknown")
            print(f"{i}. 파일 경로: {file_path}")
            print("내용:")
            print(document[:300])  # 첫 300자만 출력
            print("-" * 50)
    except Exception as e:
        print(f"전체 파일 목록 및 내용 출력 중 오류 발생: {e}")

# 파일명을 기준으로 Chroma DB에서 내용을 검색하고 반환.
def search_by_filename(file_name, vector_store):
    try:
        # Chroma DB의 모든 데이터를 가져옵니다.
        results = vector_store.get(include=["metadatas", "documents"])

        matching_files = []
        for metadata, document in zip(results["metadatas"], results["documents"]):
            if file_name in metadata.get("file_path", ""):
                matching_files.append((metadata, document))

        if matching_files:
            return matching_files[0][1]
        else:
            print(f"파일명 '{file_name}'이(가) DB에 존재하지 않습니다.")
            return None
    except Exception as e:
        print(f"파일명으로 검색 중 오류 발생: {e}")

In [442]:
github_repo_url = 'https://github.com/adunStudio/codereviewexample'
local_repo_path = "./temp_repo"

# 1. GitHub 레포지토리 클론 또는 업데이트
clone_github_repo(github_repo_url, local_repo_path)

# 2. Python 파일 찾기
python_files = find_python_files(local_repo_path)
print(f"찾은 Python 파일 수: {len(python_files)}")

# 3. Python 파일을 Chroma DB에 저장
store_python_files_in_chroma(python_files, vector_store)

#list_all_files_with_content(vector_store)

로컬에 이미 레포지토리가 존재합니다. 변경 사항을 pull합니다.
레포지토리 클론 또는 업데이트 완료!
찾은 Python 파일 수: 8
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/world.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/cat.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/text_processor.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/bird.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/animal.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/calculator.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/dog.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/main.py
Chroma DB에 모든 파일 저장 완료!


---
# [MY CODE] 6. Retriever

In [443]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [444]:
# cat.py 소스코드
input_code = search_by_filename('cat.py', vector_store)

# cat.py 소스코드와 연관된 코드 검색
retrievered_codes = retriever.invoke(input_code)

print("\n### 출처 코드 ###")
print(f'문서 개수: {len(retrievered_codes)}')
for code in retrievered_codes:
    print(f"파일: {code.metadata.get('file_path', 'N/A')}")
    print(f"내용: {code.page_content[:200]}")  # 문서 내용 일부 출력
    print('-------' * 5)


### 출처 코드 ###
문서 개수: 5
파일: ./temp_repo/cat.py
내용: from animal import Animal

class Cat(Animal):
    def __init__(self, name, age, color):
        super().__init__(name, age)
        self.color = color
        self.sound = "야옹"

    def scratch(self, 
-----------------------------------
파일: ./temp_repo/dog.py
내용: from animal import Animal

class Dog(Animal):
    def __init__(self, name, age, breed):
        super().__init__(name, age)
        self.breed = breed
        self.sound = "멍멍"

    def fetch(self, it
-----------------------------------
파일: ./temp_repo/animal.py
내용: class Animal:
    def __init__(self, name, age):
        self.name = name
        self.age = age  
        self.sound = None

    def make_sound(self):
        print(f"{self.name}이(가) 소리를 냅니다: {self.s
-----------------------------------
파일: ./temp_repo/bird.py
내용: from animal import Animal

class Bird(Animal):
    def __init__(self, name, age, wing_span):
        super().__init__(name, age)
        self.wing_span =

---
# [MY CODE] 7. PromptTemplates

In [445]:
from langchain.prompts import PromptTemplate
from langchain.prompts import load_prompt

simple_review_prompt = load_prompt('./prompts/simple_review_template.json')

input_message = ''

for code in retrievered_codes:
    input_message += f"\n\n[파일명: {code.metadata.get('file_path', 'N/A')}]\n\n"
    input_message += code.page_content

#review_prompt = review_template.format(context=input_message)
#print(review_prompt)

---
# [MY CODE] 8. TransformChain(Retriever -> PromptTemplate)

In [446]:
from langchain.chains import TransformChain

def create_input_code(inputs):
    documents = inputs["documents"]
    input_message = ""
    for doc in documents:
        input_message += f"\n\n[파일명: {doc.metadata.get('file_path', 'N/A')}]\n\n"
        input_message += doc.page_content
    return {"context": input_message}

code_mapping_chain = TransformChain(
    input_variables=["documents"],
    output_variables=["context"],
    transform=create_input_code
)

# [MY CODE] 9. 출력 파서(PydanticOutputParser)

In [447]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class Issue(BaseModel):
    issue: str = Field(description="문제 설명")
    suggestion: str = Field(description="원본 코드 첨부 및 해결 방안")

class FileReview(BaseModel):
    file_name: str = Field(description="파일명")
    issues: List[Issue] = Field(description="발견된 문제 목록")
    summary: str = Field(description="10문장 이내로 요약한 파일 리뷰")

class MultiFileReview(BaseModel):
    reviews: List[FileReview] = Field(description="여러 파일의 리뷰 결과")

pydantic_review_output_parser = PydanticOutputParser(pydantic_object=MultiFileReview)

simple_review_prompt = simple_review_prompt.partial(output_format=pydantic_review_output_parser.get_format_instructions())

---
# 10. Chain

In [448]:
openai_chain    = retriever | code_mapping_chain | simple_review_prompt | openai_llm    | pydantic_review_output_parser
anthropic_chain = retriever | code_mapping_chain | simple_review_prompt | anthropic_llm | pydantic_review_output_parser

# 입력: cat.py 소스코드
input_code = search_by_filename('cat.py', vector_store)

---
# 간단 버전
## OpenAI 응답 결과

In [449]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as callback:
    simple_openai_response = openai_chain.invoke(input_code)

    # 토큰 관련 정보 출력
    print(f"입력 토큰 수: {callback.prompt_tokens}")
    print(f"응답 토큰 수: {callback.completion_tokens}")
    print(f"총 토큰 수: {callback.total_tokens}")
    print(f"총 비용: ${callback.total_cost}")

입력 토큰 수: 1224
응답 토큰 수: 753
총 토큰 수: 1977
총 비용: $0.0006353999999999999


In [450]:
import json

# print(simple_openai_response)
# print(json.dumps(simple_openai_response, indent=2, ensure_ascii=False))
review_result = simple_openai_response
# 결과 출력
print("리뷰 결과:")
for review in review_result.reviews:
    print(f"파일명: {review.file_name}")
    for issue in review.issues:
        print(f"- 문제: {issue.issue}")
        print(f"  해결 방안: {issue.suggestion}")
    print(f"요약: {review.summary}")
    print("-" * 50)

리뷰 결과:
파일명: ./temp_repo/cat.py
- 문제: Positional argument is not used correctly in the scratch method.
  해결 방안: Replace 'self.item' with 'item' in the print statement: print('{0}이(가) {1}을(를) 긁습니다.'.format(self.name, item))
- 문제: 하드코딩된 소리 처리. 확장성이 낮음.
  해결 방안: 소리(value)를 생성자에 매개변수로 받거나, Animal 클래스에 정의하도록 만듭니다.
요약: 이 파일은 Cat 클래스를 정의하고 있으며, 상속을 잘 사용하고 있습니다. 그러나 scratch 메서드에서 잘못된 변수를 사용하고 있으며, 하드코딩된 소리 처리가 확장성을 저해하고 있습니다.
--------------------------------------------------
파일명: ./temp_repo/dog.py
- 문제: fetch 메서드에서 비효율적인 for 루프 사용.
  해결 방안: fetch 메서드의 for 루프를 제거하고, 불필요한 처리 없이 직접적으로 '가져온다'는 행동을 수행하도록 수정합니다.
- 문제: 하드코딩된 소리 처리.
  해결 방안: 소리를 생성자의 매개변수로 받아들여 확장성을 더할 수 있습니다.
요약: Dog 클래스는 Animal 클래스를 상속받아 잘 구현되어 있습니다. 그러나 fetch 메서드는 불필요한 루프를 포함하고 있으며, 소리 처리도 하드코딩 되어 있습니다.
--------------------------------------------------
파일명: ./temp_repo/animal.py
- 문제: make_sound 메서드의 디자인 문제가 있음. 무조건 '소리를 낸다'라는 템플릿 메서드 형태로 수정함이 좋음.
  해결 방안: Animal 클래스 내 make_sound를 abstract 메서드로 만들어 하위 클래스에서 구현하도록 합니다.
요약: Animal 

---
## Anthropic 응답 결과

In [451]:
simple_anthropic_response = anthropic_chain.invoke(input_code)
#print(simple_anthropic_response.content)
#print(json.dumps(simple_anthropic_response, indent=2, ensure_ascii=False))
review_result = simple_anthropic_response
# 결과 출력
print("리뷰 결과:")
for review in review_result.reviews:
    print(f"파일명: {review.file_name}")
    for issue in review.issues:
        print(f"- 문제: {issue.issue}")
        print(f"  해결 방안: {issue.suggestion}")
    print(f"요약: {review.summary}")
    print("-" * 50)

리뷰 결과:
파일명: ./temp_repo/cat.py
- 문제: scratch 메서드에서 item 파라미터를 사용하지 않고 self.item을 사용하고 있어 버그가 발생합니다.
  해결 방안: print("{0}이(가) {1}을(를) 긁습니다.".format(self.name, self.item)) -> print("{0}이(가) {1}을(를) 긁습니다.".format(self.name, item))
- 문제: 문자열 포매팅 방식이 f-string과 .format()이 혼용되어 있습니다.
  해결 방안: 모든 문자열 포매팅을 f-string으로 통일하는 것을 추천합니다:
print(f"{self.name}이(가) {item}을(를) 긁습니다.")
요약: Cat 클래스는 Animal을 상속받아 구현된 고양이 클래스입니다. 기본적인 구조는 좋으나 scratch 메서드에 버그가 있고, 문자열 포매팅이 일관성이 없습니다.
--------------------------------------------------
파일명: ./temp_repo/dog.py
- 문제: fetch 메서드에 불필요한 for 루프가 있어 성능 저하를 일으킵니다.
  해결 방안: for 루프를 제거하고 단순히 print 문만 실행하도록 수정:
def fetch(self, item):
    print(f"{self.name}이(가) {item}을(를) 가져옵니다!")
요약: Dog 클래스는 Animal을 상속받아 구현된 강아지 클래스입니다. fetch 메서드에 불필요한 지연 코드가 있습니다.
--------------------------------------------------
파일명: ./temp_repo/animal.py
- 문제: 타입 힌트가 누락되어 있어 코드의 가독성과 유지보수성이 떨어집니다.
  해결 방안: 타입 힌트 추가:
def __init__(self, name: str, age: int):
    self.name: str = name
    self.age: int = 